In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
df= pd.read_excel('combined_with_emotion12.xlsx')

In [3]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,...,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Total Interactions (weighted — Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x ),Overperforming Score,Event,Sentiment,Emotions
0,0,2308,2308.0,Tova Leigh - My Thoughts about stuff,mythoughtsaboutstuff,100044563713758,VIDEO_CREATOR,PT,European Tour Tickets NOW ON SALE! 👇🏻\nhttps:/...,2016-01-10 13:50:44,...,Photos from Tova Leigh - My Thoughts about stu...,NaN,NaN,NaN,NaN,860,-1.30,1,Pro-Israel,fear
1,1,2309,2309.0,United With Israel,unitedwithisrael,100064521497049,ORG_GENERAL,IL,🇮🇱 Stand United With Israel- Get all updates a...,2010-09-16 13:49:03,...,Hollywood Actress Amplifies Cry for Hostages H...,Actress Mayim Bialik aims to prevent the fadin...,NaN,NaN,NaN,860,5.15,1,Neutral,neutral
2,2,2310,2310.0,Eyuz Gain To Create,EyuzGaintocreate,100078050021470,DIGITAL_CREATOR,PK,🐥 BABY WHICH IS CAPABLE OF LEARNING MORE AND M...,2022-02-05 13:24:48,...,NaN,NaN,NaN,NaN,NaN,859,1.64,1,Neutral,fear
3,3,2311,2311.0,AJ+,ajplusenglish,100064529598989,MEDIA_NEWS_COMPANY,US,AJ+ is a unique digital news and storytelling ...,2014-06-09 17:11:21,...,Israeli Forces Attack Protesting Palestinians ...,NaN,NaN,NaN,NaN,859,1.01,1,Pro-Palest,fear
4,4,2312,2312.0,CNEWS,CNEWSofficiel,100064666454815,TV_CHANNEL,FR,"Toute l'info sur le canal 16 de la TNT, mais a...",2009-03-26 04:32:42,...,"«On entendait ""Dieu est le plus grand, abattez...","Le kibboutz de Be'eeri, situé proche de la ban...",NaN,NaN,NaN,858,2.43,1,Neutral,fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50466,50466,46299,NaN,CCTV,cctvcom,100044253138260,MEDIA_NEWS_COMPANY,CN,We strive to provide audiences with reports an...,40149.470845,...,Israeli military claims targeting Hamas in pre...,The Israeli military said on Sunday it had car...,NaN,NaN,NaN,16,-1.88,6,Neutral,fear
50467,50467,46300,NaN,Wall Street Journal Opinion,WSJOpinion,100064739706710,MEDIA_NEWS_COMPANY,US,Opinion & Commentary from The Wall Street Jour...,41429.596296,...,Opinion | Al Qaeda and Iran Salute the U.S. Ca...,Anti-Israel protesters in keffiyeh chic are ‘o...,NaN,NaN,NaN,16,-1.38,6,Neutral,neutral
50468,50468,46301,NaN,Rabbi Shmuley Boteach,RabbiShmuleyBoteach,100044333171299,PERSON,US,"Rabbi Shmuley, whom Newsweek & Wash Post call ...",39825.671065,...,NaN,NaN,NaN,NaN,NaN,16,-1.88,6,Pro-Israel,fear
50469,50469,46302,NaN,Rabbi Shmuley Boteach,RabbiShmuleyBoteach,100044333171299,PERSON,US,"Rabbi Shmuley, whom Newsweek & Wash Post call ...",39825.671065,...,NaN,NaN,NaN,NaN,NaN,16,-3.88,6,Pro-Israel,anger


In [4]:
propalest= df[df['Sentiment']=='Pro-Palest']
proisrael = df[df['Sentiment']=='Pro-Israel']

In [18]:
# Initialize CountVectorizer
count_vect = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
X_counts = count_vect.fit_transform(df['Message'])


In [19]:
# Initialize LDA Model
lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', max_iter=20, random_state=42)

# Fit the model
X_topics = lda_model.fit_transform(X_counts)

In [20]:
# Extract topic-word matrix
topic_word = lda_model.components_
vocab = count_vect.get_feature_names_out()

# Display the top words for each topic
n_top_words = 10
topic_summaries = []
print('Group Top Words')
print('-----', '-'*80)
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    top_words = ' '.join(topic_words)
    topic_summaries.append(top_words)
    print('  %3d %s' % (i, top_words))

Group Top Words
----- --------------------------------------------------------------------------------
    0 hamas family terrorists old held whatsapp home october hostages bodies
    1 palestine free instagram christmas real attention tell care scene muslims
    2 captive hostage new iran store org news facebook use deal
    3 minister netanyahu prime benjamin d9 d8 foreign meeting viral aviv
    4 gaza genocide israel international palestinian ceasefire people stop palestinians end
    5 eyes people world hostage time like children just today know
    6 israel pro university students protest protests hamas student jewish support
    7 rafah https com www news gaza al youtube watch camp
    8 israel gaza hamas israeli attack idf war hospital said killed
    9 palestine police protesters solidarity sea palestinian east middle support movement


In [21]:
# Get topic distribution for each document
doc_topic_distrib = lda_model.transform(X_counts)

# Assign the topic with the highest probability to each document
df['Topic'] = np.argmax(doc_topic_distrib, axis=1)

# Split the combined data back into propalest and proisrael
propalest['Topic'] = df.loc[propalest.index, 'Topic']
proisrael['Topic'] = df.loc[proisrael.index, 'Topic']
df['Topic']= df.loc[df.index, 'Topic']

C:\Users\User\AppData\Local\Temp\ipykernel_33368\4142830716.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  propalest['Topic'] = df.loc[propalest.index, 'Topic']
C:\Users\User\AppData\Local\Temp\ipykernel_33368\4142830716.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proisrael['Topic'] = df.loc[proisrael.index, 'Topic']


In [22]:
propalest[['Message','Topic']]

,Message,Topic
3,Since armed Palestinian fighters launched the ...,8
17,📷: Palestinians mourn their relatives killed i...,2
29,CNN correspondent Clarissa Ward was confronted...,8
125,Fears of a possible strike on Gaza’s Al-Quds H...,7
161,Medical and relief workers are pleading for sa...,7
...,...,...
50453,Pakistan denounces Israeli attack on a refugee...,7
50456,Join us this Wednesday May 8th 5pm-6:30pm at t...,8
50457,Canada is providing an additional $65 million ...,7
50462,"Zionists hit Rafah, ignore ICJ order",5


In [23]:
proisrael[['Message','Topic']]

,Message,Topic
0,"Watching in horror the brutal massacre, kidnap...",5
12,"We, members of the undersigned human rightsand...",5
19,"🇮🇱🇧🇷 ATTENTION: Strong images!⚠️ Amit Man, 22,...",1
20,I’m honored to have been invited to Los Angele...,5
25,#IsraelHamasWar: U.S President Joe Biden In Te...,8
...,...,...
50379,FIGHTING FOR OUR HOMELAND – DAY 214 The IDF re...,8
50380,"Israel Defense Forces: Earlier today (Sunday),...",8
50436,𝗟𝗮𝘂𝗿𝗮 𝗲𝗻𝘀𝘂𝗿𝗲𝘀 𝘀𝗼𝗹𝗱𝗶𝗲𝗿𝘀 𝗮𝗿𝗲 𝘄𝗲𝗹𝗹-𝗳𝗲𝗱 𝗮𝗻𝗱 𝘄𝗲𝗹𝗹-𝗲...,5
50468,HEAR THIS IDF COMMANDER’S BATTLEFIELD ADDRESS ...,5


In [24]:
df[['Message','Topic']]

,Message,Topic
0,"Watching in horror the brutal massacre, kidnap...",5
1,Actress Mayim Bialik aims to prevent the fadin...,8
2,#Gaza #viral #attack #israel #trending #eyuzga...,8
3,Since armed Palestinian fighters launched the ...,8
4,An alarming testimony,0
...,...,...
50466,The Israeli military said on Sunday it had car...,8
50467,U.S. students who have “All Eyes on Rafah” but...,5
50468,HEAR THIS IDF COMMANDER’S BATTLEFIELD ADDRESS ...,5
50469,If Egypt dares abrogate the peace for which Is...,4
